In [ ]:
!pip install git+https://github.com/Navu45/image-merging.git

In [ ]:
import gradio as gr
import torch

from models.image_merging import StableDiffusionImageMergingPipeline

pipe = StableDiffusionImageMergingPipeline.from_pretrained(
    "lambdalabs/sd-image-variations-diffusers", revision="v2.0",
    torch_dtype=torch.float16, use_safetensors=True
)
pipe.to("cuda")
pipe.enable_xformers_memory_efficient_attention()


def create_img(source_image, merge_image) -> list:
    generator = torch.Generator(device="cuda").manual_seed(42)
    image = pipe(source_image, merge_image, generator=generator, guidance_scale=3).images[0]
    return image


with gr.Blocks() as demo:
    with gr.Tab("Merge two images"):
        with gr.Row():
            source_image_input = gr.Image()
            merge_image_input = gr.Image()
        with gr.Row():
            image_output = gr.Image()
        image_button = gr.Button("Generate merged image!")
    image_button.click(create_img, inputs=[source_image_input, merge_image_input], outputs=image_output)

demo.launch(debug=True)
